## Library Imports

In [1]:
import json, os, requests, re, string

# Setting up Memory

In [2]:
MEMORY_FILE = "memory.json"

if not os.path.exists(MEMORY_FILE):
    with open(MEMORY_FILE, "w") as f:
        json.dump([], f)


# ---------- MEMORY SYSTEM ----------
def store_memory(keywords, value):
    with open(MEMORY_FILE, "r") as f:
        data = json.load(f)

    for k in keywords:
        data.append({"key": k.lower(), "value": value})

    with open(MEMORY_FILE, "w") as f:
        json.dump(data, f, indent=2)


def search_memory(key):
    key = key.lower().strip()
    with open(MEMORY_FILE, "r") as f:
        data = json.load(f)

    matches = [m["value"] for m in data if key in m["key"]]
    return matches if matches else "No memory found."


# Helper Functions

In [3]:
# ---------- CALCULATOR ----------
def calculate(expr):
    try:
        return eval(expr, {"__builtins__": None}, {})
    except:
        return "Could not calculate."


In [4]:
# ---------- NORMALIZERS ----------
def clean_coin_name(name):
    name = name.lower().strip()
    name = name.translate(str.maketrans("", "", string.punctuation))

    mapping = {
        "bitcoin": "bitcoin",
        "btc": "bitcoin",
        "ethereum": "ethereum",
        "eth": "ethereum"
    }
    return mapping.get(name, name)


# Tools Definition

In [5]:
# ---------- TOOLS ----------
def weather_api(city):
    try:
        url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
        geo = requests.get(url).json()
        if "results" not in geo:
            return "City not found."

        lat = geo["results"][0]["latitude"]
        lon = geo["results"][0]["longitude"]

        weather = requests.get(
            f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
        ).json()

        temp = weather["current_weather"]["temperature"]
        return f"The temperature in {city} is {temp}°C."

    except:
        return "Weather unavailable."


def dictionary_api(word):
    try:
        word = word.strip()
        data = requests.get(f"https://api.dictionaryapi.dev/api/v2/entries/en/{word}").json()
        meaning = data[0]["meanings"][0]["definitions"][0]["definition"]
        return meaning
    except:
        return "No definition found."


def crypto_price(coin):
    try:
        url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin}&vs_currencies=usd"
        data = requests.get(url).json()
        price = data[coin]["usd"]
        return f"${price}"
    except:
        return "Coin not found."


# Router Function

In [6]:
# ---------- ROUTER ----------
def agent(query):
    q = query.lower().strip()

    # MEMORY STORE
    if q.startswith("remember"):
        fact = query.split("that")[-1].strip()
        keywords = fact.lower().split()
        store_memory(keywords, fact)
        return f"Remembered: {fact}"

    # MEMORY RECALL
    if "remember" in q and "what" in q:
        key = q.replace("what do you remember about", "").strip()
        key = key.replace("what do you remember", "").strip()
        return search_memory(key)

    # DICTIONARY TOOL
    if q.startswith("define") or "meaning of" in q or "definition of" in q:
        word = q.replace("define", "").replace("meaning of", "").replace("definition of", "").strip()
        return dictionary_api(word)

    # CRYPTO PRICE
    if "price" in q and ("bitcoin" in q or "btc" in q or "ethereum" in q or "eth" in q):
        coin = clean_coin_name(q.split("price")[-1].strip())
        return crypto_price(coin)

    # WEATHER TOOL
    if "weather" in q:
        city = q.replace("weather in", "").replace("weather", "").strip()
        return weather_api(city)

    if "temperature in" in q:
        city = q.replace("temperature in", "").strip()
        return weather_api(city)

    # CALCULATOR
    if re.search(r"\d+\s*[\+\-\*/]\s*\d+", q):
        expr = re.search(r"[0-9+\-*/(). ]+", q).group()
        return calculate(expr)

    return "I'm a small AI agent, but I'm learning!"


# Gradio Interface

In [10]:
!pip install gradio

In [7]:
import gradio as gr

In [8]:
def chat_interface(message, history):
    response = agent(message)
    return response

ui = gr.ChatInterface(
    fn=chat_interface,
    title="LilAgent",
    description="Ask me anything — weather, dictionary, crypto, memory, calculator!"
)

ui.launch()


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://519504672cac67acb6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
